In [1]:
from resnet_ecg.utils import one_hot,get_batches
from resnet_ecg.ecg_preprocess import ecg_preprocessing
from resnet_ecg.densenet import DenseNet

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from utils import extract_basic_features

import wfdb
import os
import wfdb.processing as wp
import matplotlib.pyplot as plt
from scipy import signal
from utils import find_noise_features, extract_basic_features
import shutil

import time
from lightgbm import LGBMClassifier
from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from xgboost import XGBClassifier


In [3]:
import os
import warnings
warnings.filterwarnings("ignore")
import scipy.io as sio
train_dataset_path = os.getcwd()+"/Train/"
val_dataset_path = os.getcwd()+"/Val/"

In [4]:
train_files = os.listdir(train_dataset_path)
train_files.sort()
val_files = os.listdir(val_dataset_path)
val_files.sort()

In [5]:
labels = pd.read_csv("reference.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def read_data_seg(data_path, split = "Train",preprocess=False):
    """ Read data """

    # Fixed params
    n_class = 2
    n_steps = 2560

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    #label_path = os.path.join(data_path, "reference.txt")
    #labels = pd.read_csv(label_path, sep='\t',header = None)
    #labels = pd.read_csv("reference.csv")

    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)
    #posix = len(split) + 5

    # Initiate array
    list_of_channels = []
    X = np.zeros((len(channel_files), n_steps, n_channels))
    i_ch = 0
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:])):
        #channel_name = fil_ch[:-posix]
        #dat_ = pd.read_csv(os.path.join(path_signals,fil_ch), delim_whitespace = True, header = None)
        #print(fil_ch)
        
        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        
        ecg_channels = np.zeros((n_steps, n_channels))
          
        for i_n,ch_name in enumerate(channel_name):
            
            if ch_name in channel_mid_name:
                mid_ind = int(ecg[ch_name].T.shape[0]/2)
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],2560).T 
            elif ch_name in channel_post_name:
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],2560).T
            else:
                ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],2560).T
                
            #ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,2560).T

            if preprocess:
                data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,2560), 'sym8', 8, 3, 256)

                ecg_channels[:,i_n] = data[0]#ecg['data']
            else:
                pass
                #ecg_channels[:,i_n] = ecg_channels[:,i_n]
                
        X[i_ch,:,:] = ecg_channels

    # Return 
    return X

In [7]:
def read_data_sample(data_path, split = "Train",preprocess=False):
    """ Read data """

    # Fixed params
    n_class = 2
    n_steps = 2560

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    #label_path = os.path.join(data_path, "reference.txt")
    #labels = pd.read_csv(label_path, sep='\t',header = None)

    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)
    #posix = len(split) + 5

    # Initiate array
    list_of_channels = []
    X = np.zeros((len(channel_files), n_steps, n_channels))
    i_ch = 0
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1',  'II', 'aVR', 'V3', 'III', 'V5']
    
    #shape_ecg = []
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:])):
        #channel_name = fil_ch[:-posix]
        #dat_ = pd.read_csv(os.path.join(path_signals,fil_ch), delim_whitespace = True, header = None)
        #print(fil_ch)
        
        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        
        
        
        ecg_channels = np.zeros((n_steps, n_channels))
        
        for i_n,ch_name in enumerate(channel_name[:]):
            
            ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,2560).T
            
            #shape_ecg.append(ecg[ch_name].shape[1])
            
            if preprocess:
                data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,2560), 'sym8', 8, 3, 256)

                ecg_channels[:,i_n] = data[0]#ecg['data']
            else:
                pass
                #ecg_channels[:,i_n] = ecg_channels[:,i_n]
                
        X[i_ch,:,:] = ecg_channels

    # Return 
    return X#shape_ecg#

In [8]:
train_x = read_data_sample(os.getcwd(),preprocess=True)#read_data_sample(os.getcwd(),preprocess=True) 

6500it [01:45, 61.38it/s]


In [9]:
sum1_label=0
sum2_label=0
sum3_label=0
sum4_label=0
sum5_label=0
sum6_label=0
sum7_label=0
bin_label = np.zeros((6500,9))
for i in range(labels.shape[0]):
    label_nona = labels.loc[i].dropna()
    if label_nona.shape[0] == 2:
        sum1_label+=1
    if label_nona.shape[0] == 3:
        sum2_label+=1
    if label_nona.shape[0] == 4:
        sum3_label+=1
    if label_nona.shape[0] == 5:
        sum4_label+=1
    if label_nona.shape[0] == 6:
        sum5_label+=1
    if label_nona.shape[0] == 7:
        sum6_label+=1
    if label_nona.shape[0] == 8:
        sum7_label+=1
    for j in range(1,label_nona.shape[0]):
        bin_label[i,int(label_nona[j])]=1

In [10]:
bin_label.shape

(6500, 9)

In [11]:
train_x.shape

(6500, 2560, 12)

In [12]:
from keras.utils import to_categorical
from keras.optimizers import SGD,Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from sklearn.model_selection import StratifiedKFold

In [13]:
class Config(object):
    def __init__(self):
        self.conv_subsample_lengths = [1, 2, 1, 2, 1, 2, 1, 2]
        self.conv_filter_length = 32
        self.conv_num_filters_start = 12
        self.conv_init = "he_normal"
        self.conv_activation = "relu"
        self.conv_dropout = 0.5
        self.conv_num_skip = 2
        self.conv_increase_channels_at = 2
        self.batch_size = 32#128
        self.input_shape = [2560, 12]#[1280, 1]
        self.num_categories = 2

    @staticmethod
    def lr_schedule(epoch):
        lr = 0.1
        if epoch >= 20 and epoch < 40:
            lr = 0.01
        if epoch >= 40:
            lr = 0.001
        print('Learning rate: ', lr)
        return lr

In [14]:

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5

session = tf.Session(config=config)


KTF.set_session(session )

model = DenseNet(input_shape=(2560,12), nb_classes=9, depth=10, growth_rate=16,
                          dropout_rate=0.1, bottleneck=False, compression=0.5)

Creating DenseNet 0.0.1
#############################################
Dense blocks: 3
Layers per dense block: [2, 2, 2]
#############################################
Instructions for updating:
Colocations handled automatically by placer.
block::: 0
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
block::: 1
nb_channels:: Tensor("concatenate_6/concat:0", shape=(?, 160, 60), dtype=float32)


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2560, 12)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 2560, 16)     6144        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 2560, 16)     64          conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 2560, 16)     0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv1d_2 (

In [16]:
import keras.backend as K

def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


In [17]:
from sklearn.model_selection import train_test_split
X_tr, X_vld, lab_tr, lab_vld = train_test_split(train_x, labels['label1'], 
                                        stratify = labels['label1'], #test_size=0.2,
                                        shuffle=True, random_state = 123)

In [18]:
lab_tr.index.tolist()[:2]

[4788, 208]

In [19]:
def preprocess_y(y,num_class=9):
    bin_label = np.zeros((y.shape[0],num_class))
    for i in range(y.shape[0]):
        label_nona = labels.loc[lab_tr.index.tolist()[i]].dropna()
        for j in range(1,label_nona.shape[0]):
            bin_label[i,int(label_nona[j])]=1
    return bin_label

In [20]:
y_tr = preprocess_y(lab_tr)

In [21]:
y_vld = preprocess_y(lab_vld) 

In [22]:
checkpointer = ModelCheckpoint(filepath='weights_best_simple_model_1.hdf5', 
                            monitor='val_fmeasure',verbose=1, save_best_only=True, mode='max')
reduce = ReduceLROnPlateau(monitor='val_fmeasure',factor=0.5,patience=2,verbose=1,min_delta=1e-4,mode='max')
''' 
model.compile(optimizer = 'adam',
           loss='binary_crossentropy',
           metrics=['accuracy',fmeasure,recall,precision])

epochs = 20

history = model.fit_generator(train_generator,
       validation_data = val_generator,
       epochs=epochs,
       callbacks=[checkpointer,reduce],
       verbose=1)
'''
pass

In [23]:
from keras import backend as K

#2156+224+672+654+180+826+534+504+1953

def weighted_loss(labels,logits):
    logits_log = (-1) * K.log(logits)
    num_class = logits.shape[-1]
    
    y_0 = 0.8 * labels[:,0:1]
    y_1 = 1.2 * labels[:,1:2]
    y_2 = 1.0 * labels[:,2:3]
    y_3 = 1.0 * labels[:,3:4]
    y_4 = 1.2 * labels[:,4:5]
    y_5 = 1.0 * labels[:,5:6]
    y_6 = 1.0 * labels[:,6:7]
    y_7 = 1.0 * labels[:,7:8]
    y_8 = 0.8 * labels[:,8:]
    
    weight_labels = K.concatenate([y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8],axis=1)
    loss = K.sum(logits_log * weight_labels,axis=1)
    
    loss = K.mean(loss)
    
    return loss

In [24]:
def add_compile(model, config):
    
    optimizer = SGD(lr=config.lr_schedule(0), momentum=0.9)#Adam()#
    model.compile(loss='binary_crossentropy',#weighted_loss,#'binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy',fmeasure,recall,precision])   

In [25]:
cw = {0:1,1:3,2:3,3:2,4:9,5:3,6:3,7:9,8:1}

In [26]:
config = Config()
add_compile(model, config)

model_name = 'resnet12.h5'
earlystop = EarlyStopping(
            monitor='val_fmeasure',#'val_categorical_accuracy',
            patience=10,
            )
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_categorical_accuracy', mode='max',
                             save_best_only='True')

lr_scheduler = LearningRateScheduler(config.lr_schedule)

callback_lists = [checkpointer,reduce]#[checkpointer,earlystop,lr_scheduler]
#[checkpoint, earlystop,lr_scheduler] 

history = model.fit(x=X_tr, y=y_tr, batch_size=32, epochs=50,  #class_weight=cw,#'auto',
          verbose=1, validation_data=(X_vld, y_vld), callbacks=callback_lists )

Learning rate:  0.1
Instructions for updating:
Use tf.cast instead.
Train on 4875 samples, validate on 1625 samples
Epoch 1/50
4875/4875 [==============================] - 15s 3ms/step - loss: 0.3456 - acc: 0.8913 - fmeasure: 0.5996 - recall: 0.5438 - precision: 0.7120 - val_loss: 0.5880 - val_acc: 0.8225 - val_fmeasure: 0.2159 - val_recall: 0.1858 - val_precision: 0.2593

Epoch 00001: val_fmeasure improved from -inf to 0.21587, saving model to weights_best_simple_model_1.hdf5
Epoch 2/50
4875/4875 [==============================] - 7s 1ms/step - loss: 0.2303 - acc: 0.9252 - fmeasure: 0.6557 - recall: 0.5444 - precision: 0.8292 - val_loss: 0.7660 - val_acc: 0.8213 - val_fmeasure: 0.2306 - val_recall: 0.2033 - val_precision: 0.2676

Epoch 00002: val_fmeasure improved from 0.21587 to 0.23059, saving model to weights_best_simple_model_1.hdf5
Epoch 3/50
4875/4875 [==============================] - 7s 1ms/step - loss: 0.2024 - acc: 0.9307 - fmeasure: 0.6873 - recall: 0.5811 - precision: 0.84

In [30]:
test_x = read_data_sample(os.getcwd(),split='Val',preprocess=True)#read_data_sample(os.getcwd(),split='Val',preprocess=True)#

500it [00:08, 61.03it/s]


In [36]:
test_y = model.predict(test_x)  #predict_proba

In [37]:
test_y

array([[9.8503208e-01, 5.1888824e-04, 8.5613132e-04, ..., 3.2211900e-02,
        2.3009121e-02, 2.7737021e-04],
       [9.8081112e-02, 1.2540817e-04, 4.1021705e-03, ..., 1.4446229e-02,
        8.8787079e-04, 7.2325170e-03],
       [1.7760694e-03, 3.1007361e-01, 3.2027692e-02, ..., 6.4637882e-01,
        5.6105852e-03, 5.0534213e-01],
       ...,
       [1.8537045e-05, 4.6483576e-03, 3.2123178e-02, ..., 3.3958137e-02,
        3.6358833e-05, 5.6140423e-03],
       [6.6468149e-02, 2.0886660e-03, 7.9625750e-01, ..., 5.1576585e-02,
        9.8766983e-03, 2.2523215e-01],
       [7.8547132e-01, 1.2442470e-03, 3.0868053e-03, ..., 3.9200875e-01,
        4.6236217e-03, 1.8393993e-03]], dtype=float32)

## 1 select best_threshold based on train data reference

In [48]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
import csv

In [50]:
x_tr_y = model.predict(train_x)#X_tr

threshold = np.arange(0.1,0.9,0.1)

out = x_tr_y
y_test = bin_label#y_tr

acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold: ",best_threshold)

y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])

y_pred 

y_test

hamming_loss(y_test,y_pred)

best_threshold:  [0.6 0.5 0.3 0.2 0.2 0.4 0.4 0.4 0.4]


0.03011965811965812

In [51]:
classes = [0,1,2,3,4,5,6,7,8]

y_pred = [[1 if test_y[i,j]>=best_threshold[j] else 0 for j in range(test_y.shape[1])] 
          for i in range(len(test_y))]
pred=[]
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])

with open('answers2.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

## 2 select a single threshold

In [29]:
import csv
thred = 0.2

pred = []
for i in range(test_y.shape[0]):
    
    try:
        pred_list = list(np.hstack(np.argwhere(test_y[i]>thred)))
    except ValueError:
        print(" ValueError !!! ")
        pred_list = ['']
        
    pred.append(pred_list)
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

### remove 0 or others

In [31]:
import csv
thred = 0.2#0.25

pred = []
for i in range(test_y.shape[0]):
    if np.argmax(test_y[i]) == 0:
        pred.append([0])
        continue
    try:
        pred_list = list(np.hstack(np.argwhere(test_y[i]>thred)))
    except ValueError:
        pred_list = ['']
        
    if 0 in pred_list:
        if np.argmax(test_y[i]) != 0:
            pred_list.remove(0)
            
    pred.append(pred_list)
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [ ]:
X_tr.shape

## k-fold

In [ ]:
seed = 42
NFOLDS = 5
history = []
y_test_pred = []

train = X_tr #x_tr
test = test_x
train_label =  y_tr

kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
kf = kfold.split(train, train_label)

config = Config()
add_compile(model, config)

model_name = 'resnet12.h5'
earlystop = EarlyStopping(
            monitor='val_categorical_accuracy',
            patience=10,
            )
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_categorical_accuracy', mode='max',
                             save_best_only='True')

lr_scheduler = LearningRateScheduler(config.lr_schedule)

callback_lists = [checkpoint, earlystop,lr_scheduler]

history = model.fit(x=X_tr, y=y_tr, batch_size=32, epochs=50,
          verbose=1, validation_data=(X_vld, y_vld), callbacks=[checkpointer,reduce])



In [ ]:
seed = 42
NFOLDS = 5
history = []#np.zeros(NFOLDS)
y_test_pred = []#np.zeros(NFOLDS)

train = train_x #x_tr
#test = x_test[:400]
train_label =  bin_label#y_tr[1].values

kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
kf = kfold.split(train, train_label)

train_data_use = train
#test_data_use = test
pre_train_data_use = pre_x_tr

for i, (train_fold, validate) in enumerate(kf):
    print('fold: ',i, ' training')
    np.random.shuffle(train_fold)
    np.random.shuffle(validate)
    #print(train_fold)
    #print(validate)
    
    # original data
    X_train0, X_val0, label_train0, label_val0 = train_data_use[train_fold], train_data_use[validate], \
                                                train_label[train_fold], train_label[validate]
    # preprocessed data
    X_train1, X_val1, label_train1, label_val1 = pre_train_data_use[train_fold], pre_train_data_use[validate], \
                                                train_label[train_fold], train_label[validate]
        
    X_train = np.concatenate([X_train0 , X_train1],axis=0)
    X_val = np.concatenate([X_val0 , X_val1],axis=0)
    label_train = np.concatenate([label_train0 , label_train1],axis=0)
    label_val = np.concatenate([label_val0 , label_val1],axis=0)
    
    #only original data for train
    #X_train, X_val, label_train, label_val= X_train0, X_val0, label_train0, label_val0
    
    print(X_train.shape)
    print(X_val.shape)
    
    print(label_train.shape)
    print(label_val.shape)
    
    # convert one-hot
    one_hot_y_tr = one_hot(label_train)
    one_hot_y_val  = one_hot(label_val)
    
    config = Config()
    add_compile(model, config)
    model_name = 'merge_densenet_fold{0}.h5'.format(i+1)
    earlystop = EarlyStopping(
                monitor='val_categorical_accuracy',
                patience=20,
                )
    checkpoint = ModelCheckpoint(filepath=model_name,
                                 monitor='val_categorical_accuracy', mode='max',
                                 save_best_only='True')

    lr_scheduler = LearningRateScheduler(config.lr_schedule)
    callback_lists = [checkpoint, earlystop,lr_scheduler]
    history.append( model.fit(x=X_train, y=one_hot_y_tr, batch_size=config.batch_size, epochs=100,
              verbose=1, validation_data=(X_val, one_hot_y_val), callbacks=callback_lists))
    
    #y_test_pred.append( model.predict(test_data_use))
    #break